## ChatGPT

This is just some messing around.

Docs:

* how to upload images: https://platform.openai.com/docs/guides/vision
* types of models: https://stackoverflow.com/questions/75774873/openai-api-error-this-is-a-chat-model-and-not-supported-in-the-v1-completions
* token limits: https://platform.openai.com/settings/organization/limits
* models: https://platform.openai.com/docs/models/model-endpoint-compatibility

In [1]:
#!pip install openai

In [ ]:
#dir_api = '/Users/jnaiman/Dropbox/Paper_JCDL2025/chatgpt_api/' # where to store API results

dir_api = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/' #store API results for example-hists

key_file = '/Users/jnaiman/.openai/key.txt'

jsons_dir = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/' # directory where jsons created with figure are stored
imgs_dir = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/imgs/' # where images are stored

# for saving temp images for reading in
tmp_dir = '/Users/jnaiman/Downloads/tmp/'

img_format = 'jpeg'

In [ ]:
import openai
import base64
from openai import OpenAI
from PIL import Image
import numpy as np
import json
import re
import pickle
import os
from glob import glob

from sys import path
path.append('../')
from utils.llm_utils import parse_qa, load_image

In [ ]:
# def load_image(image_path, fac=0.5):
#     img = Image.open(image_path).convert('RGB')
#     new_size = np.round(np.array(img.size)*fac).astype('int')
#     img = img.resize(new_size, Image.Resampling.LANCZOS)
#     #img = np.array(img)
#     #with open(image_path, "rb") as image_file:
#     img.save('/Users/jnaiman/Downloads/tmp/tmp_img.png')
#     with open('/Users/jnaiman/Downloads/tmp/tmp_img.png','rb') as image_file:
#         #return base64.b64encode(img).decode("utf-8")
#         return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
# # parsing
# def parse_qa(level_parse, plot_level, qa, j, types, 
#              partials = ['persona', 'context','question', 'format']):
#     keys_tmp = list(j[level_parse][plot_level].keys())
#     keys = []
#     for k in keys_tmp:
#         if '(' in k:
#             k = k.split('(')[0].rstrip()
#         keys.append(k)
    
#     keys = np.unique(keys).tolist()

#     dirs_partials = {}
    
#     for k in keys:
#         v = ''
#         kk = ''
#         for t in types:
#             if k + " " + t in j[level_parse][plot_level]:
#                 v = j[level_parse][plot_level][k + " " + t]
#                 #kk = k + " " + t
#                 break
#         if v == '':
#             v = j[level_parse][plot_level][k]
#             #kk = k
#         if 'A' in v: # no plot
#             if type(v['A']) == type({}):
#                 ans = list(v['A'].values())[0]
#             else:
#                 ans = v['A']
#             if type(v['Q']) == type({}):
#                 que = list(v['Q'].values())[0]
#             else:
#                 que = v['Q']
#             # get other elements
#             for p in partials:
#                 dirs_partials[p] = v[p]
#         else: # plotX
#             for kk,vv in v.items():
#                 ans = vv['A']
#                 while type(ans) == type({}):
#                     ans_1 = list(ans.values())[0]
#                     if 'plot' in list(ans.keys())[0]:
#                         ans = ans_1
#                         break
#                     ans = ans_1
#                 que = vv['Q']
#                 # get other elements
#                 for p in partials:
#                     dirs_partials[p] = vv[p]
#         out = {'Q':que, 'A':ans, 'Level':level_parse, 'type':plot_level, 'Response':""}
#         for kp,vp in dirs_partials.items():
#             out[kp] = vp
#         # if there is a plot
#         if 'plot' in list(v.keys())[0]:
#             out['plot number'] = list(v.keys())[0]
#             #import sys; sys.exit()
#         qa.append(out)
#     return qa


In [ ]:
def get_img_json_pair(img_path, json_path, dir_api, 
                      tmp_dir = '/Users/jnaiman/Downloads/tmp/',
                      fac = 1.0, 
                      restart = False, verbose = True):
    """
    img_path : where image file is stored
    json_path : where json path is stored
    dir_api : where we can look for prior, saved pickles, if applicable
    fac : do we want to downsize the image? IF so, set to < 1

    returns: encoded image, full json from creation run, error
      encoded image and full json are empty strings if error is True
    """
    #print('on', iFile, 'of', iMax)
    err = False
    base_file = json_path.split('/')[-1].removesuffix('.json')
    if os.path.exists(dir_api + base_file + '.pickle') and not restart:
        if verbose: print('have file already:', dir_api + base_file + '.pickle')
        return '','', True
    # do we have it?
    try:
        #image_path = '/Users/jnaiman/Downloads/data_full_v2/Picture'+str(int(iFile))+'.png'
        encoded_image = load_image(img_path, fac=fac, tmp_dir=tmp_dir)
        # get questions
        with open(json_path,'r') as f:
            j = json.loads(f.read())
            j = json.loads(j)
    except Exception as e:
        if verbose: print('[ERROR]:', str(e))
        err = True
        return '','', True
    
    return encoded_image, j, err

In [ ]:
def send_to_chatgpt(question_list, client, image_path, encoded_image,
                    model ="gpt-4o-mini", 
                    test_run = True, fac=1.0, 
                    verbose=True):
    #model="gpt-4",
    #model="gpt-4o",
    #model ="gpt-4o-mini",
    #model ="gpt-3.5-turbo",
    #model="gpt-3.5-turbo-instruct",

    # query chatgpt
    #responses = []

    #try:
    iFac = 1.0
    #for question_list in qa:
    success = False
    iFac = 1
    #['persona', 'context','question', 'format']
    while not success:
        try:
            #question = question_list['Q']
            question = question_list['context'] + " " + question_list['question'] + " " + question_list['format']
            # lowercase the first word, just in case
            question = question.lstrip() # no whitespace
            question = question[0].lower() + question[1:]
            if verbose: print('   on question:',question)
            # Prepare the API request
            prompt = f"I am going to show you an image. Here is the image: [Image: {encoded_image}]. Now, {question}"
            prompt_save = f"I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, {question}"
            ##question_list['prompt'] = prompt
            
            if not test_run:
                # Send the request to the GPT-4o API
                response = client.chat.completions.create(
                    model = model,
                    messages=[
                        {"role": "system", "content": question_list['persona']},
                        {"role":"user", "content": [
                            {
                            "type": "text",
                            "text": prompt
                            },
                            {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{encoded_image}"
                            }
                            }
                        ]
                        }
                    ]
                )
                success = True
            else:
                success = True
        except Exception as e:
            print(e)
            new_fac = fac/iFac
            print('new fac = ', new_fac)
            encoded_image = load_image(image_path,fac=new_fac, tmp_dir=tmp_dir)
            iFac += 1
    
    if not test_run:
        # Get the response from the API
        answer = response.choices[0].message.content
        # format answer
        answer_format = answer.replace("```json\n",'').replace("\n```",'')
        try:
            question_list['Response'] = json.loads(answer_format)
        except:
            question_list['Response'] = answer_format
            question_list['Error'] = 'JSON formatting'
        question_list['Response String'] = answer_format
        success = True
    else:
        question_list['Response'] = 'TEST RUN'
        question_list['Response String'] = 'TEST RUN'

    return question_list, prompt_save

In [122]:
def parse_for_errors(qa, verbose=True, print_what = 'prompt'):
    """
    print_what : set to "prompt" to print full prompt, or "question" for just the question
    """
    # try to fix
    for level in qa:
        noErr = False
        if 'Error' in level:
            noErr = True
            try:
                iters = []
                for x in re.finditer(r'//(\s*)(.*)(\s*)\n', level['Response']):
                    #print(x)
                    iters.append(level['Response'][x.span()[0]:x.span()[1]])
                
                response = level['Response']
                for i in iters:
                    response = response.replace(i,'')
                level['Response'] = response
                del level['Error']
            except:
                noErr = False
                print('********')
                print('couldnt fix:', level['Response'])
                print('********')
                pass
    
        try:
            if type(level['Response']) != type({}):
                level['Response'] = json.loads(level['Response'])
        except:
            pass
    
        if not noErr and verbose:
            if print_what == 'question':
                print('Q:', level['Q'])
            elif print_what == 'prompt':
                print('Q:', level['prompt'])
            print('ChatGPT A:', level['Response'])
            print('Real A:   ', level['A'])

            print('')
            #if 'Error' in level:
            #    import sys; sys.exit()
    return qa

In [100]:
# setup
with open(key_file,'r') as f:
    api_key = f.read()

client = OpenAI(
  api_key=api_key.strip(),  # this is also the default, it can be omitted
)

In [101]:
jsons_to_parse = glob(jsons_dir + '/*.json')
jsons_to_parse[:3]

['/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/nclust_3_trial9.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/nclust_5_trial3.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/nclust_2_trial0.json']

In [ ]:
iMax = 2
verbose = False
test_run = False # run w/o actually pinging openai
restart = False
model ="gpt-4o-mini"

fac = 0.5
for ijson,json_path in enumerate(jsons_to_parse):
    if ijson >= iMax:
        continue

    print('on', ijson, 'of', iMax)

    # get image and base json
    img_path = imgs_dir + json_path.split('/')[-1].removesuffix('.json') + '.' + img_format
    encoded_image, base_json, err = get_img_json_pair(img_path, json_path, dir_api, 
                                                      fac=fac, restart=restart,
                                                      tmp_dir=tmp_dir)

    if err:
        continue


    ###### create QA ########
    qa = []
    
    for k,v in base_json['VQA']['Level 1']['Figure-level questions'].items():
        out = {'Q':v['Q'], 'A':v['A'], 'Level':'Level 1', 'type':'Figure-level questions', 'Response':""}
        qa.append(out)
    
    # what kinds?
    types = ['(words + list)', '(words)']
    
    # get uniques
    level_parse = 'Level 1'
    plot_level = 'Plot-level questions'
    qa = parse_qa(level_parse, plot_level, qa, base_json['VQA'], types)
    
    level_parse = 'Level 2'
    plot_level = 'Plot-level questions'
    qa = parse_qa(level_parse, plot_level, qa, base_json['VQA'], types)
    
    level_parse = 'Level 3'
    plot_level = 'Plot-level questions'
    qa = parse_qa(level_parse, plot_level, qa, base_json['VQA'], types)

    responses = []
    for question_list in qa:
        response, prompt = send_to_chatgpt(question_list, client, img_path, encoded_image,
                    model = model, 
                    test_run = test_run)
        responses.append(response)
        question_list['prompt'] = prompt

    # parse for errors
    qa = parse_for_errors(qa)

    # dump to file
    if not test_run:
        with open(dir_api + json_path.split('/')[-1].removesuffix('.json')+ '.pickle', 'wb') as ff:
            pickle.dump([qa, model], ff)
    else:
        print('Would store at:', dir_api + json_path.split('/')[-1].removesuffix('.json')+ '.pickle')


on 0 of 2
have file already: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/nclust_3_trial9.pickle
on 1 of 2
   on question: how many bars are there in the specified figure panel? Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.
   on question: what are the maximum data values in this figure panel?  Please format the output as a json as {"maximum x":""} for this figure panel, where the "maximum" value should be a float, calculated from the data values used to create the plot.
   on question: what are the mean data values in this figure panel?  Please format the output as a json as {"mean x":""} for this figure panel, where the "mean" value should be a float, calculated from the data values used to create the plot.
   on question: what are the median data values in this figure panel?  Please format the output as a json as {"median x":""} for this figure panel, where the "median" value should be

## Look at data

Check out one, if you wanna:

In [129]:
pickles = glob(dir_api + '*.pickle')
pickles[:5]

['/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/nclust_5_trial3.pickle',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/nclust_3_trial9.pickle']

In [130]:
ifile = 0
with open(pickles[ifile], 'rb') as f:
    qa_in = pickle.load(f)[0]

In [132]:
print(pickles[ifile])
print('*********')
for qa_pairs in qa_in:
    print('Prompt:', qa_pairs['prompt'])
    print('   Real A:', qa_pairs['A'])
    print('ChatGPT A:', qa_pairs['Response'])
    print('')

/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/nclust_5_trial3.pickle
*********
Prompt: I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, how many bars are there in the specified figure panel? Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.
   Real A: 50
ChatGPT A: {'nbars': 20}

Prompt: I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, what are the maximum data values in this figure panel?  Please format the output as a json as {"maximum x":""} for this figure panel, where the "maximum" value should be a float, calculated from the data values used to create the plot.
   Real A: 0.531999058363311
ChatGPT A: {'maximum x': 0.5}

Prompt: I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, what are the mean data values in this figure panel?  Please format the output as a json as {"mean x":""} for this